In [226]:
############################################################################
# Использование API Google SpreadSheets
############################################################################


############################################################################
# Описание API
#
# API для Google SpreadSheets, позволяет программно выполнять действия, 
# например, отправлять файлы.
# Документация по API SpreadSheets: 
# https://docs.gspread.org/en/latest/
# https://gspread.readthedocs.io/en/latest/
# https://developers.google.com/sheets/api/reference/rest/v4/spreadsheets
###########################################################################

In [227]:
# Импортируем библиотеки

import pandas as pd
import datetime
import os
import random
import sys
import requests
import json
from urllib.parse import urlencode
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [228]:
# Укажем адрес электронной почты и путь к нашему файлу с учетными данными

my_mail = 'user@gmail.com'
path_to_credential = 'testapi.json' 

In [229]:
# Укажем имя таблицы в Google SpreadSheets

table_name = 'TestAPI'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

In [230]:
# Авторизация

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)
gs = gspread.authorize(credentials)

In [231]:
# Выберем первый лист (вкладку) в документе

work_sheet = gs.open(table_name)
sheet1 = work_sheet.sheet1

In [232]:
# Получим данные в формате списков Python

data = sheet1.get_all_values()
data

[['ID', 'Name', 'Cart'],
 ['1', 'User234', '45669875'],
 ['2', 'User567', '5245698'],
 ['3', 'User123', '654885214'],
 ['4', 'User985', '214585'],
 ['5', 'User098', '92223655']]

In [233]:
# Получим заголовки из данных

headers = data.pop(0)

In [234]:
# Создадим датафрейм из найшей таблицы с данными в Google SpreadSheets

df = pd.DataFrame(data, columns = headers)
df.head()

,ID,Name,Cart
0,1,User234,45669875
1,2,User567,5245698
2,3,User123,654885214
3,4,User985,214585
4,5,User098,92223655


In [235]:
# Создание новой таблицы

table_name = 'New spreadsheet'
sheet = gs.create(table_name)

In [236]:
# Сортировка пользователей по имени

df_new = df.sort_values('Name')
df_new

,ID,Name,Cart
4,5,User098,92223655
2,3,User123,654885214
0,1,User234,45669875
1,2,User567,5245698
3,4,User985,214585


In [237]:
# Настройка прав доступа к новой таблице

sheet.share(my_mail, perm_type='user', role='writer')

In [238]:
# Экспорт датафрэйма в созданную таблицу

sheet = 'Master'
d2g.upload(df_new, table_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'Master' id:1609043479>

In [ ]:
# Получение ссылки на созданный документ

sheet = gs.create(table_name)
spreadsheet_url = "https://docs.google.com/spreadsheets/d/%s" % sheet.id
spreadsheet_url